# Reverse engineer molecules: drug application

#### July 2024 version

In [ ]:
# CHANGE TO YOUR DIRECTORY PATH
path_directory = "C:/Users/meyerp/Documents/INRAE/Diophantine/Enumération/github/signature"

# packages
import numpy as np
import os

import pandas as pd
import sys
import time

np.set_printoptions(threshold=sys.maxsize)

os.chdir(path_directory)

from rdkit.Chem.Descriptors import ExactMolWt
from rdkit import Chem
from rdkit import RDLogger

RDLogger.DisableLog("rdApp.*")

# Reverse engineer molecules from Morgan
from signature.enumerate_signature import enumerate_molecule_from_signature, enumerate_signature_from_morgan
from signature.enumerate_utils import reduced_fingerprint, test_sol_ECFP, test_sol_ECFP_reduced, test_sol_sig
from signature.Signature import MoleculeSignature
from signature.signature_alphabet import (
    load_alphabet,
    morgan_vector_from_signature,
    sanitize_molecule,
    SignatureAlphabet,
    signature_from_smiles,
)
from signature.utils import read_csv, read_tsv

In [ ]:
# Alphabet

# file_alphabet = "./datasets/MetaNetX_weight_500_fcharge_Alphabet_radius_2_nBits_2048_smarts_full" # smarts r2 fcharge
file_alphabet = "./datasets/MetaNetX_weight_500_fcharge_Alphabet_radius_2_boundary_nBits_2048_full" # smiles r2 boundary fcharge
# Get alphabet
Alphabet = load_alphabet(file_alphabet)
Alphabet.print_out()

- FDA drug molecules

In [ ]:
file_smiles = "./datasets/FDA_Approved_structures"
# Load Smiles file
H, D = read_csv(file_smiles)
print(H, D.shape[0])
Smiles = np.asarray(list(set(D[:, 0])))
print(f"Number of smiles: {len(Smiles)}")

- preprocess of drug Smiles

In [ ]:
# sanitize molecules
drug_molecules = []
indices_to_keep = []
for i in range(D.shape[0]):
    smi = D[i, 1]
    mol = Chem.MolFromSmiles(smi)
    _, smi_san = sanitize_molecule(mol, formalCharge=True)
    if len(smi_san) != 0:
        indices_to_keep.append(i)
    drug_molecules.append(smi_san)
D = np.insert(D, D.shape[1], drug_molecules, axis=1)
D = D[indices_to_keep, :]

# sort D by string length
last_column = D[:, 2]
lengths = np.array([len(s) for s in last_column])
sorted_indices = np.argsort(lengths)
D = D[sorted_indices]

- ECFP reduced => Signature => Smiles

In [ ]:
max_nbr_partition = int(1e5)
max_nbr_recursion = 1e5
max_nbr_solution = 1001
repeat = 10

print(f"ID | smi | Nsig | Nmol | FoundMol")
for i in range(120, D.shape[0]):
    # preparation
    smi = D[i, 2]
    #print(i, "|", smi)

    mol = Chem.MolFromSmiles(smi)
    ms = MoleculeSignature(mol, radius=Alphabet.radius, neighbor=True, use_smarts=Alphabet.use_smarts, nbits=False, boundary_bonds=Alphabet.boundary_bonds, map_root=True, legacy=False)
    sig = ms.as_deprecated_string(morgan=False, root=False, neighbors=True)
    morgan = reduced_fingerprint(smi, radius=Alphabet.radius, useFeatures=False)

    # ecfp_red => signature(s)
    Alphabet.nBits = 2048
    Ssig, partition_timeout, ct_solve = enumerate_signature_from_morgan(
        morgan, Alphabet, max_nbr_partition=max_nbr_partition, method="partitions", verbose=False
    )
    
    # signature(s) => molecule(s)
    Smol, Nsig = set(), 0
    Alphabet.nBits = 0
    for j in range(len(Ssig)):
        smol, recursion_timeout = enumerate_molecule_from_signature(
            Ssig[j],
            Alphabet,
            smi,
            max_nbr_recursion=max_nbr_recursion,
            max_nbr_solution=max_nbr_solution,
            repeat=repeat,
            verbose=False,
        )
        Smol = Smol | set(smol)
    
    Alphabet.nBits = 2048
    Smolfinal = []
    for smi2 in Smol:
        if test_sol_ECFP_reduced([smi, smi2], Alphabet=Alphabet):
            Smolfinal.append(smi2)
    foundmol = smi in Smolfinal

    Smolfinal.append(smi)
    Smolfinal = list(set(Smolfinal))
    
    drugs = [s for s in Smolfinal if s in list(D[:, 2])]
    
    if len(drugs) > 1:
        print(f"{i} | {smi} | {len(Ssig)} | {len(Smolfinal)} | {int(foundmol)}")
        print("nb_drug", len(drugs), "Smolfinal", drugs)

# Create database from drugbank

- function to export html from url

In [ ]:
import requests

def get_html_content(url, verbose=False):
    try:
        # Send a GET request to the URL
        response = requests.get(url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            return response.content  # Return the HTML content of the page
        else:
            print(f"Failed to retrieve HTML content. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

# Example usage:
drugbank_url = "https://go.drugbank.com/drugs/DB00002"
html_content = get_html_content(drugbank_url)

if html_content:
    print("HTML content retrieved successfully.")
    # Now you can pass this html_content to your extraction function or use it as needed
    extract_pubchem_substance_id(html_content)
else:
    print("Failed to retrieve HTML content from the specified URL.")

- function to extract pubchem id from html_content

In [ ]:
from bs4 import BeautifulSoup

def extract_pubchem_substance_id(html_content, verbose=False):
    try:
        # Parse the HTML content
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Find the element containing the PubChem Substance ID
        pubchem_substance_element = soup.find('dt', {'id': 'pubchem-substance'})
        
        if pubchem_substance_element:
            # Get the next sibling element which contains the PubChem Substance ID link
            pubchem_substance_id_element = pubchem_substance_element.find_next_sibling('dd')
            
            if pubchem_substance_id_element:
                # Extract the PubChem Substance ID
                pubchem_substance_id_link = pubchem_substance_id_element.find('a')
                
                if pubchem_substance_id_link:
                    pubchem_substance_id = pubchem_substance_id_link.text.strip()
                    if verbose:
                        print(f"PubChem Substance ID: {pubchem_substance_id}")
                    return pubchem_substance_id
                elif verbose:
                    print("PubChem Substance ID link not found in the HTML content")
            elif verbose:
                print("PubChem Substance ID element not found in the HTML content")
        elif verbose:
            print("PubChem Substance ID element not found in the HTML content")
    
    except Exception as e:
        print(f"Error: {e}")
    
    return -1
    
# Example HTML content (replace with your actual HTML content)
html_content = '''
<a target="_blank" rel="noopener" href="http://www.genome.jp/dbget-bin/www_bget?drug:D03455">D03455</a></dd><dt class="col-md-4 col-sm-5" id="pubchem-substance">PubChem Substance</dt><dd class="col-md-8 col-sm-7"><a target="_blank" rel="noopener" href="http://pubchem.ncbi.nlm.nih.gov/summary/summary.cgi?sid=46507042">46507042</a></dd>
'''

# Call the function with the provided HTML content
extract_pubchem_substance_id(html_content)

- loop to create a list of pubchem id from drugbank id

In [ ]:
pubchem_ids = []
for n in range(1, 10):
    drugbank_id = "DB" + "0"*(5-len(str(n)))+str(n)
    drugbank_url = "https://go.drugbank.com/drugs/" + drugbank_id
    html_content = get_html_content(drugbank_url)
    pubchem_id = extract_pubchem_substance_id(html_content)
    if pubchem_id != -1:
        pubchem_ids.append(pubchem_id)

In [ ]:
pubchem_ids

- smiles

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def get_page_text(url):
    try:
        # Set up Selenium Chrome web driver
        driver_service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=driver_service)
        
        # Open the URL in Selenium WebDriver
        driver.get(url)
        
        # Wait for the page to fully load
        driver.implicitly_wait(10)  # Adjust the timeout as needed
        
        # Get the entire visible text from the webpage
        page_text = driver.find_element(By.TAG_NAME, 'body').text
        
        # Close the Selenium WebDriver
        driver.quit()
        
        return page_text
    
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

# Example usage:
url = "https://pubchem.ncbi.nlm.nih.gov/compound/46507042#section=Canonical-SMILES&fullscreen=true"
page_text = get_page_text(url)

if page_text:
    print(f"Page text:\n{page_text}")
else:
    print("Failed to retrieve page text.")

In [ ]:
def extract_smiles_from_pubchem(url):
    page_text = get_page_text(url)
    page_content = page_text.split("\n")
    indice = page_content.index("Canonical SMILES")
    smi = page_content[indice + 1]
    return smi

url = "https://pubchem.ncbi.nlm.nih.gov/compound/46507042#section=Canonical-SMILES&fullscreen=true"
extract_smiles_from_pubchem(url)

In [ ]:
drugs_smiles = []
for pubchem_id in pubchem_ids:
    #print(pubchem_id)
    url = "https://pubchem.ncbi.nlm.nih.gov/compound/" + pubchem_id + "#section=Canonical-SMILES&fullscreen=true"
    smiles = extract_smiles_from_pubchem(url)
    #print(smiles)
    drugs_smiles.append(smiles)

#### old functions (to del)

- Fonctions to search if a molecule is a drug

In [ ]:
import requests

def is_drug_or_not(smiles, verbose=False):
    base_url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/'
    url = base_url + f'{smiles}/cids/JSON'
    
    drugbank_id = -1
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes
        
        data = response.json()
        if 'IdentifierList' in data and 'CID' in data['IdentifierList']:
            cid = data['IdentifierList']['CID'][0]
            
            # Now, check if the compound is categorized as a drug
            drug_url = f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/DrugBankID/JSON'
            drug_response = requests.get(drug_url)
            drug_data = drug_response.json()
            
            if 'PropertyTable' in drug_data and 'DrugBankID' in drug_data['PropertyTable']:
                drugbank_id = drug_data['PropertyTable']['DrugBankID']
                if verbose:
                    if drugbank_id:
                        print(f"Molecule with SMILES '{smiles}' is registered as a drug in PubChem (DrugBank ID: {drugbank_id}, CID: {cid})")
                    else:
                        print(f"Molecule with SMILES '{smiles}' is not registered as a drug in PubChem (CID: {cid})")
            else:
                if verbose:
                    print(f"Molecule with SMILES '{smiles}' is not registered as a drug in PubChem (CID: {cid})")
        else:
            if verbose:
                print(f"Molecule with SMILES '{smiles}' not found in PubChem")
    
    except requests.exceptions.RequestException as e:
        if verbose:
            print(f"Error: {e}")
    
    return drugbank_id, cid

In [ ]:
import requests

def check_pubchem_cid_in_drugbank(pubchem_cid):
    try:
        # Construct the DrugBank REST API URL
        base_url = 'https://go.drugbank.com/structures/'
        url = base_url + f'pubchem_cid/{pubchem_cid}.json'
        
        # Send a GET request to DrugBank API
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad status codes
        
        data = response.json()
        
        # Check if the molecule is found in DrugBank
        if 'drugbank_id' in data:
            drugbank_id = data['drugbank_id']
            print(f"Compound with PubChem CID '{pubchem_cid}' is registered as a drug in DrugBank (DrugBank ID: {drugbank_id})")
        else:
            print(f"Compound with PubChem CID '{pubchem_cid}' is not registered as a drug in DrugBank")
    
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")

# Example usage:
pubchem_cid = '12620'  # Replace with the PubChem CID you want to check
check_pubchem_cid_in_drugbank(pubchem_cid)

In [ ]:
def nb_drug_in_a_set(S, verbose=False):
    drugs_id = set()
    for s in S:
        drug_id = is_drug_or_not(s, verbose=verbose)
        if drug_id != -1:
            drugs_id = drugs_id.add(drug_id)
    return len(drugs_id)

In [ ]:
# Example usage:
smiles = 'CCCCCCCCCCCCCCCCCCCCCCO'
print(check_if_drugbank_drug(smiles))

In [ ]:
# Example usage:
smiles = 'CCCCCCCCCCCCCCCCCCCCCCO'
print(is_drug_or_not(smiles))

S = ['CCCCCCCCCCCCCCCCCCCCCCO']
print(nb_drug_in_a_set(S, verbose=False))